In [14]:
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re
from tqdm import tqdm
from IPython.display import HTML
import base64
import zipfile
import os
import PIL.Image as Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torchvision.transforms as transforms
from torchvision import datasets

is_trained = False

def create_download_link(title="Download CSV file", filename="experiment/kaggle.csv"):
    df = pd.read_csv(filename)
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')


def sample_image():
    bird_classes = os.listdir('../input/bird_dataset/bird_dataset/train_images/')
    sample_class = np.random.choice(bird_classes)
    training_path = '../input/bird_dataset/bird_dataset/train_images/'
    sample_image = np.random.choice(os.listdir(training_path+sample_class))
    path_sample_image = training_path + sample_class + '/' + sample_image
    x = plt.imread(path_sample_image)
    return x

def train(epoch, model, train_loader, record_loss=True):
    """
        Train the global defined model.
        Warnings:
            model: is a global variable here
        Input:
            epoch: The index number of the epoch (This is NOT the number of epochs)
            record_loss: Record the loss evolution for the model
    """
    # loss recording
    loss_evolution = []

    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        criterion = torch.nn.CrossEntropyLoss(reduction='elementwise_mean')
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args_log_interval == 0:
            loss_value = loss.data.item()
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss_value))
        if record_loss:
            loss_evolution.append(loss_value)

    if record_loss:
        return loss_evolution

def get_model(path_experiment, number=-1):
    if number < 0:
        model_names = os.listdir(path_experiment)
        print(model_names)
        model_numbers = []
        for name in model_names:
            regex = r'(\d+)\.pth'
            pattern = re.compile(regex)
            match = re.search(pattern, name)
            if match is not None:
                model_numbers.append(int(match.group(1)))
        number = np.max(model_numbers)
    return f'./experiment/model_{number}.pth'


def validation(model, val_loader):
    """
        Compute validation score
        Warnings:
            As in train(), the model is a global variable definition
        Input:
        Output:
            validation_loss <float>
            accuracy <float>

    """
    model.eval()
    validation_loss = 0
    correct = 0
    for data, target in val_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        output = model(data)
        # sum up batch loss
        criterion = torch.nn.CrossEntropyLoss(reduction='elementwise_mean')
        validation_loss += criterion(output, target).data.item()
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    validation_loss /= len(val_loader.dataset)
    accuracy = 100. * correct / len(val_loader.dataset)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        validation_loss, correct, len(val_loader.dataset),
        accuracy))

    return validation_loss, accuracy

def adjust_learning_rate(learning_rate, optimizer, epoch, schedule=[150, 225], gamma=0.1):
    if epoch in schedule:
        learning_rate *= gamma
        for param_group in optimizer.param_groups:
            param_group['lr'] = learning_rate
    return learning_rate


args_data = 'bird_dataset/'
training_path = os.path.join(args_data, 'train_images')
bird_classes = os.listdir(training_path)
sample_class = np.random.choice(bird_classes)

# parameters
n_classes = len(bird_classes)
args_seed = 1
args_log_interval = 10
args_experiment = 'experiment'

# Create experiment folder
if not os.path.isdir(args_experiment):
    os.makedirs(args_experiment)

## Model

In [11]:
transform_train = transforms.Compose([
#     transforms.RandomCrop(64, padding=4),
    transforms.Resize((299, 299)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
#     transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

args_batch_size = 64
args_epochs = 200
args_lr = 0.1
args_weight_decay = 1e-4
args_momentum = 0.9

            
import torch.nn as nn
import torch.utils.model_zoo as model_zoo

__all__ = ['InceptionV4', 'inceptionv4']

model_urls = {
    'inceptionv4': 'https://s3.amazonaws.com/pytorch/models/inceptionv4-58153ba9.pth'
}

class BasicConv2d(nn.Module):

    def __init__(self, in_planes, out_planes, kernel_size, stride, padding=0):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_planes, out_planes, kernel_size=kernel_size, stride=stride, padding=padding, bias=False) # verify bias false
        self.bn = nn.BatchNorm2d(out_planes, eps=0.001, momentum=0, affine=True)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x

class Mixed_3a(nn.Module):

    def __init__(self):
        super(Mixed_3a, self).__init__()
        self.maxpool = nn.MaxPool2d(3, stride=2)
        self.conv = BasicConv2d(64, 96, kernel_size=3, stride=2)

    def forward(self, x):
        x0 = self.maxpool(x)
        x1 = self.conv(x)
        out = torch.cat((x0, x1), 1)
        return out

class Mixed_4a(nn.Module):

    def __init__(self):
        super(Mixed_4a, self).__init__()

        self.block0 = nn.Sequential(
            BasicConv2d(160, 64, kernel_size=1, stride=1),
            BasicConv2d(64, 96, kernel_size=3, stride=1)
        )

        self.block1 = nn.Sequential(
            BasicConv2d(160, 64, kernel_size=1, stride=1),
            BasicConv2d(64, 64, kernel_size=(1,7), stride=1, padding=(0,3)),
            BasicConv2d(64, 64, kernel_size=(7,1), stride=1, padding=(3,0)),
            BasicConv2d(64, 96, kernel_size=(3,3), stride=1)
        )

    def forward(self, x):
        x0 = self.block0(x)
        x1 = self.block1(x)
        out = torch.cat((x0, x1), 1)
        return out

class Mixed_5a(nn.Module):

    def __init__(self):
        super(Mixed_5a, self).__init__()
        self.conv = BasicConv2d(192, 192, kernel_size=3, stride=2)
        self.maxpool = nn.MaxPool2d(3, stride=2)

    def forward(self, x):
        x0 = self.conv(x)
        x1 = self.maxpool(x)
        out = torch.cat((x0, x1), 1)
        return out

class Inception_A(nn.Module):

    def __init__(self):
        super(Inception_A, self).__init__()
        self.block0 = BasicConv2d(384, 96, kernel_size=1, stride=1)

        self.block1 = nn.Sequential(
            BasicConv2d(384, 64, kernel_size=1, stride=1),
            BasicConv2d(64, 96, kernel_size=3, stride=1, padding=1)
        )

        self.block2 = nn.Sequential(
            BasicConv2d(384, 64, kernel_size=1, stride=1),
            BasicConv2d(64, 96, kernel_size=3, stride=1, padding=1),
            BasicConv2d(96, 96, kernel_size=3, stride=1, padding=1)
        )

        self.block3 = nn.Sequential(
            nn.AvgPool2d(3, stride=1, padding=1, count_include_pad=False),
            BasicConv2d(384, 96, kernel_size=1, stride=1)
        )

    def forward(self, x):
        x0 = self.block0(x)
        x1 = self.block1(x)
        x2 = self.block2(x)
        x3 = self.block3(x)
        out = torch.cat((x0, x1, x2, x3), 1)
        return out

class Reduction_A(nn.Module):

    def __init__(self):
        super(Reduction_A, self).__init__()
        self.block0 = BasicConv2d(384, 384, kernel_size=3, stride=2)

        self.block1 = nn.Sequential(
            BasicConv2d(384, 192, kernel_size=1, stride=1),
            BasicConv2d(192, 224, kernel_size=3, stride=1, padding=1),
            BasicConv2d(224, 256, kernel_size=3, stride=2)
        )
        
        self.block2 = nn.MaxPool2d(3, stride=2)

    def forward(self, x):
        x0 = self.block0(x)
        x1 = self.block1(x)
        x2 = self.block2(x)
        out = torch.cat((x0, x1, x2), 1)
        return out

class Inception_B(nn.Module):

    def __init__(self):
        super(Inception_B, self).__init__()
        self.block0 = BasicConv2d(1024, 384, kernel_size=1, stride=1)
        
        self.block1 = nn.Sequential(
            BasicConv2d(1024, 192, kernel_size=1, stride=1),
            BasicConv2d(192, 224, kernel_size=(1,7), stride=1, padding=(0,3)),
            BasicConv2d(224, 256, kernel_size=(7,1), stride=1, padding=(3,0))
        )

        self.block2 = nn.Sequential(
            BasicConv2d(1024, 192, kernel_size=1, stride=1),
            BasicConv2d(192, 192, kernel_size=(7,1), stride=1, padding=(3,0)),
            BasicConv2d(192, 224, kernel_size=(1,7), stride=1, padding=(0,3)),
            BasicConv2d(224, 224, kernel_size=(7,1), stride=1, padding=(3,0)),
            BasicConv2d(224, 256, kernel_size=(1,7), stride=1, padding=(0,3))
        )

        self.block3 = nn.Sequential(
            nn.AvgPool2d(3, stride=1, padding=1, count_include_pad=False),
            BasicConv2d(1024, 128, kernel_size=1, stride=1)
        )

    def forward(self, x):
        x0 = self.block0(x)
        x1 = self.block1(x)
        x2 = self.block2(x)
        x3 = self.block3(x)
        out = torch.cat((x0, x1, x2, x3), 1)
        return out

class Reduction_B(nn.Module):

    def __init__(self):
        super(Reduction_B, self).__init__()

        self.block0 = nn.Sequential(
            BasicConv2d(1024, 192, kernel_size=1, stride=1),
            BasicConv2d(192, 192, kernel_size=3, stride=2)
        )

        self.block1 = nn.Sequential(
            BasicConv2d(1024, 256, kernel_size=1, stride=1),
            BasicConv2d(256, 256, kernel_size=(1,7), stride=1, padding=(0,3)),
            BasicConv2d(256, 320, kernel_size=(7,1), stride=1, padding=(3,0)),
            BasicConv2d(320, 320, kernel_size=3, stride=2)
        )

        self.block2 = nn.MaxPool2d(3, stride=2)

    def forward(self, x):
        x0 = self.block0(x)
        x1 = self.block1(x)
        x2 = self.block2(x)
        out = torch.cat((x0, x1, x2), 1)
        return out

class Inception_C(nn.Module):

    def __init__(self):
        super(Inception_C, self).__init__()
        self.block0 = BasicConv2d(1536, 256, kernel_size=1, stride=1)
        
        self.block1_0 = BasicConv2d(1536, 384, kernel_size=1, stride=1)
        self.block1_1a = BasicConv2d(384, 256, kernel_size=(1,3), stride=1, padding=(0,1))
        self.block1_1b = BasicConv2d(384, 256, kernel_size=(3,1), stride=1, padding=(1,0))
        
        self.block2_0 = BasicConv2d(1536, 384, kernel_size=1, stride=1)
        self.block2_1 = BasicConv2d(384, 448, kernel_size=(3,1), stride=1, padding=(1,0))
        self.block2_2 = BasicConv2d(448, 512, kernel_size=(1,3), stride=1, padding=(0,1))
        self.block2_3a = BasicConv2d(512, 256, kernel_size=(1,3), stride=1, padding=(0,1))
        self.block2_3b = BasicConv2d(512, 256, kernel_size=(3,1), stride=1, padding=(1,0))
        
        self.block3 = nn.Sequential(
            nn.AvgPool2d(3, stride=1, padding=1, count_include_pad=False),
            BasicConv2d(1536, 256, kernel_size=1, stride=1)
        )

    def forward(self, x):
        x0 = self.block0(x)
        
        x1_0 = self.block1_0(x)
        x1_1a = self.block1_1a(x1_0)
        x1_1b = self.block1_1b(x1_0)
        x1 = torch.cat((x1_1a, x1_1b), 1)

        x2_0 = self.block2_0(x)
        x2_1 = self.block2_1(x2_0)
        x2_2 = self.block2_2(x2_1)
        x2_3a = self.block2_3a(x2_2)
        x2_3b = self.block2_3b(x2_2)
        x2 = torch.cat((x2_3a, x2_3b), 1)

        x3 = self.block3(x)

        out = torch.cat((x0, x1, x2, x3), 1)
        return out

class InceptionV4(nn.Module):

    def __init__(self, num_classes=1001):
        super(InceptionV4, self).__init__()
        self.features = nn.Sequential(
            BasicConv2d(3, 32, kernel_size=3, stride=2),
            BasicConv2d(32, 32, kernel_size=3, stride=1),
            BasicConv2d(32, 64, kernel_size=3, stride=1, padding=1),
            Mixed_3a(),
            Mixed_4a(),
            Mixed_5a(),
            Inception_A(),
            Inception_A(),
            Inception_A(),
            Inception_A(),
            Reduction_A(), # Mixed_6a
            Inception_B(),
            Inception_B(),
            Inception_B(),
            Inception_B(),
            Inception_B(),
            Inception_B(),
            Inception_B(),
            Reduction_B(), # Mixed_7a
            Inception_C(),
            Inception_C(),
            Inception_C(),
            nn.AvgPool2d(8, count_include_pad=False)
        )
        self.classif = nn.Linear(1536, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classif(x) 
        return x


def inceptionv4(pretrained=False):
    r"""InceptionV4 model architecture from the
    `"Inception-v4, Inception-ResNet..." <https://arxiv.org/abs/1602.07261>`_ paper.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = InceptionV4()
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['inceptionv4']))
    return model

model = inceptionv4(pretrained=True)
model.aux_logit=False
in_features = model.classif.in_features
model.fc = nn.Linear(in_features, n_classes)

## main.py

In [15]:
use_cuda = torch.cuda.is_available()
if use_cuda:
    model.cuda()
torch.manual_seed(args_seed)

train_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(args_data + '/train_images',
                         transform=transform_train),
    batch_size=args_batch_size, shuffle=True, num_workers=0)

val_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(args_data + '/val_images',
                         transform=transform_test),
    batch_size=args_batch_size, shuffle=False, num_workers=0)

optimizer = optim.SGD(model.parameters(), lr=args_lr, momentum=args_momentum, weight_decay=args_weight_decay)
# optimizer = torch.optim.Adam(model.parameters(), lr=args_lr)

In [ ]:
# Go through epochs and record loss on training set and accuracy on
train_loss_evolution = []
valid_loss_evolution = []
valid_accuracy_evolution = []

if not is_trained:
    for epoch in range(1, args_epochs + 1):
        args_lr = adjust_learning_rate(args_lr, optimizer, epoch)
        loss_evolution = train(epoch, model, train_loader, record_loss=True)
        validation_loss, accuracy = validation(model, val_loader)
        # Record
        train_loss_evolution.append(loss_evolution)
        valid_loss_evolution.append(validation_loss)
        valid_accuracy_evolution.append(accuracy)

        # Store the last 2 models
        if epoch > args_epochs-2:
            model_file = args_experiment + '/model_' + str(epoch) + '.pth'
            torch.save(model.state_dict(), model_file)
            print('\nSaved model to ' + model_file + '. You can run `python evaluate.py --model ' + model_file + '` to generate the Kaggle formatted csv file')

#### Visualise loss and accuracy evolutions

In [ ]:
if not is_trained:
    plt.figure(figsize=(15,15))

    plt.subplot('311')
    plt.title('Evolution of the training loss')
    plt.plot(np.mean(np.array(train_loss_evolution), axis=1), alpha=0.4, label='mean')
    plt.plot(np.mean(np.array(train_loss_evolution), axis=1) + 3*np.std(np.array(train_loss_evolution), axis=1), 'rs--', alpha=0.4 ,label='std_bound')
    plt.plot(np.mean(np.array(train_loss_evolution), axis=1) - 3*np.std(np.array(train_loss_evolution), axis=1), 'rs--', alpha=0.4)
    plt.boxplot(np.array(train_loss_evolution).T)
    plt.xlabel('epochs')
    plt.ylabel('validation loss')
    plt.legend()
    plt.grid()

    plt.subplot('312')
    plt.title('Validation loss evolution')
    plt.plot(np.array(valid_loss_evolution), 'xk')
    plt.plot(np.array(valid_loss_evolution), 'k--', alpha=0.4)

    plt.grid()

    plt.subplot('313')
    plt.title('Validation accuracy evolution')
    plt.plot(np.array(valid_accuracy_evolution), 'xk')
    plt.plot(np.array(valid_accuracy_evolution), 'k--', alpha=0.4)
    plt.grid()

    plt.show()

## Evaluate

In [16]:
args_outfile = os.path.join(args_experiment, 'kaggle.csv')
# args_model = get_model(args_experiment)

# state_dict = torch.load(args_model)
# model = Net()
# model.load_state_dict(state_dict)
# model.eval()
# if use_cuda:
#     model.cuda()

test_dir = args_data + '/test_images/mistery_category'


output_file = open(args_outfile, "w")
output_file.write("Id,Category\n")
for f in tqdm(os.listdir(test_dir)):
    if 'jpg' in f:
        data = transform_test(pil_loader(test_dir + '/' + f))
        data = data.view(1, data.size(0), data.size(1), data.size(2))
#         if use_cuda:
#             data = data.cuda()
        output = model(data)
        pred = output.data.max(1, keepdim=True)[1]
        output_file.write("%s,%d\n" % (f[:-4], pred))

output_file.close()

print("Succesfully wrote " + args_outfile + ', you can upload this file to the kaggle competition website')

# Check if output_file is available
print('kaggle.csv' in os.listdir('./experiment/'))
print(os.listdir('experiment'))

# create a link to download the dataframe
create_download_link('experiment/kaggle.csv')



100%|██████████| 517/517 [12:16<00:00,  1.43s/it]

Succesfully wrote experiment/kaggle.csv, you can upload this file to the kaggle competition website
True
['model_3.pth', 'model_2.pth', 'model_1.pth', 'model_5.pth', 'model_4.pth', 'model_6.pth', 'model_7.pth', 'kaggle.csv', 'model_10.pth', 'model_9.pth', 'model_8.pth']


In [ ]:
data = Variable (torch.rand(2,3,299,299))
outs = model(data)